# Automated Report Builder and Emailer
### ABB #1 - Session 1

Code authored by: Shaw Talebi

### imports

In [1]:
import csv
import pandas as pd

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

from top_secret import app_password

### load data

#### read in emails

In [2]:
# load contact info
filename = "data/emails.csv"
recipients_list = []

# read CSV file and populate the recipients list
with open(filename, mode='r') as csvfile:
    csv_reader = csv.DictReader(csvfile)
    for row in csv_reader:
        name = row["Name"].split(" ")[0]
        email = row["Email"]
        recipients_list.append((name, email))

print(recipients_list)

[('Shaw', 'shaw@example.com'), ('Luca', 'luca@example.com'), ('Elena', 'elena@example.com')]


In [3]:
# recipients_list = [('Shaw', 'shawhintalebi@gmail.com')]

#### read survey results

In [4]:
filename = "data/survey.csv"
df_survey = pd.read_csv(filename)

In [5]:
df_survey.head()

,What is your current level of experience with AI/ML?,How many years of programming experience do you have?,Which programming language(s) are you most comfortable with?
0,1-3 years,4-6 years,Python
1,1-3 years,4-6 years,Python
2,<1 year,<1 year,Python
3,4-6 years,7+ years,Python
4,<1 year,7+ years,"PHP, Javascript"


In [6]:
# response count
class_size = 43
num_responses = len(df_survey)
print(num_responses)

23


In [7]:
# AI experience distribution
ai_experience = df_survey.iloc[:,0].value_counts()
print(ai_experience)

What is your current level of experience with AI/ML?
<1 year      14
1-3 years     7
4-6 years     1
7+ years      1
Name: count, dtype: int64


In [8]:
# coding experience distribution
coding_experience = df_survey.iloc[:,1].value_counts()
print(coding_experience)

How many years of programming experience do you have?
7+ years     9
1-3 years    8
<1 year      4
4-6 years    2
Name: count, dtype: int64


In [9]:
# coding language distribution

# define coding languages
lang_list = ['Python', 'PHP', 'JavaScript', 'Bash', 'C++', 'Java', 'TypeScript', 'goLang', 'C#', 'R', 'Fortran', 'Algol', 'Basic', 'SQL', 'HTML', 'CSS']

# create a dictionary to store the count for each language
lang_dict = {}

# count occurrences of each language in the Series
for lang in lang_list:
    lang_dict[lang] = df_survey.iloc[:,2].str.lower().str.count(r'\b' + lang.lower() + r'\b').sum()

# convert the counts to a DataFrame for easier readability
df_lang = pd.DataFrame(list(lang_dict.items()), columns=['Language', 'Count']).set_index('Language').sort_values(by='Count', ascending=False)
print(df_lang)

            Count
Language         
Python         18
C++             7
JavaScript      4
PHP             3
Bash            2
Java            2
TypeScript      1
goLang          1
R               1
Fortran         1
Algol           1
Basic           1
SQL             1
HTML            1
CSS             1
C#              0


### construct message

In [10]:
def remove_first_last_lines(text):
    lines = text.splitlines()
    return "\n".join(lines[1:-1])

def remove_first_two_lines(text):
    lines = text.splitlines()
    return "\n".join(lines[2:])

In [11]:
subject = "The results are in..."
body = lambda student_name : f"""Hey {student_name},

I wanted to share the pre-course survey results before our first session tomorrow.

We had {num_responses} responses out of the {class_size} enrolled ({int(100*round(num_responses/class_size,2))}% response rate).

Here's a breakdown of this cohort's incoming experience 👇

AI/ML Experience:
{remove_first_last_lines(str(ai_experience))}

Programming Experience:
{remove_first_last_lines(str(coding_experience))}

Top Languages:
{remove_first_two_lines(str(df_lang[:4]))}

Looking forward to meeting you all tomorrow!

-Shaw

P.S. If you haven't completed the pre-course survey yet, you can access it here: https://forms.gle/YQ5DbbovRUf5Utvc8"""

In [12]:
subject = "The results are in..."
print(body("Test"))

Hey Test,

I wanted to share the pre-course survey results before our first session tomorrow.

We had 23 responses out of the 43 enrolled (53% response rate).

Here's a breakdown of this cohort's incoming experience 👇

AI/ML Experience:
<1 year      14
1-3 years     7
4-6 years     1
7+ years      1

Programming Experience:
7+ years     9
1-3 years    8
<1 year      4
4-6 years    2

Top Languages:
Python         18
C++             7
JavaScript      4
PHP             3

Looking forward to meeting you all tomorrow!

-Shaw

P.S. If you haven't completed the pre-course survey yet, you can access it here: https://forms.gle/YQ5DbbovRUf5Utvc8


### send emails

In [13]:
# Define your email credentials and recipient
sender_email = "your email here"
password = app_password

# Connect to the server
with smtplib.SMTP("smtp.gmail.com", 587) as server:
    server.starttls()  # Start TLS encryption
    server.login(sender_email, password)  # Log into email account

    # Send the email to each recipient with personalized content
    for name, email in recipients_list:
        # Set up the MIME
        message = MIMEMultipart()
        message["From"] = sender_email
        message["To"] = email
        message["Subject"] = subject
        
        # Email body with personalization
        personalized_body = body(name)
        message.attach(MIMEText(personalized_body, "plain"))

        # Send the email
        server.sendmail(sender_email, email, message.as_string())
        print(f"Email sent successfully to {name} at {email}!")

Email sent successfully to Shaw at shaw@example.com!
Email sent successfully to Luca at luca@example.com!
Email sent successfully to Elena at elena@example.com!
